# Symbol changes extraction using dedicated service - Python

### Overview
Name and ticker changes can be seen as a corporate actions event or a static data update. In this sample we are dealing with changes automated updates as a corporate actions event.

A name change or a ticker change can be handled as a corporate actions event and occurs in various use-cases such as: 
* a company decide to change its name and the issued secuities from from an old name to a current name.
* in mergers and acquisitions, old names are combined to a new one

Symbol change should be spread all along the investment, risk and data management workflow. Here are few examples of immediate consequent symbol updates: 
* data collection robots
* watchlists and analytics dahsboards
* strategy building and simulation bots
* indices components if the symbol belongs to one or more indices
* risk models

### Inputs/outputs
Changes sample is designed to help you searching and requesting symbol changes for a given instrument.
* It takes instrument specification parameter(s) as per input filter and returns *Changes*.

### Services used
This sample uses *gRPC requests* in order to retrieve the corporate actions changes from the hosted service. The queried endpoint in this script are:
* *ChangesService*: to directly retrieve changes data objects from the server

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.corporate_actions.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    
***

# Run symbol changes sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas

In [ ]:
pip install systemathics.apis --pre

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.services.corporate_actions.v1.changes_pb2 as changes
import systemathics.apis.services.corporate_actions.v1.changes_pb2_grpc as changes_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Prepare API requests
The following code snippets retrieve authentication token to be used in upcomming API requests:

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Retrieve changes

#### 3.1 Equity selection

In [ ]:
ticker = 'ISCB'
exchange = 'XNYS'

#### 3.2 Create and process request

In [ ]:
# create the symbol changes request
request = changes.ChangesRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        #instantiate the symbol changes service
        service = changes_service.ChangesServiceStub(channel)
        
        # process the symbol changes request
        response = service.Changes(
            request = request, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### 3.3 Retrieve changes

In [ ]:
# define a method that creates a pandas dataframe out of symbol changes response
def get_changes(response):
    date = [datetime(d.date.year, d.date.month, d.date.day).date() for d in response.data]
    old_symbol = [d.old_symbol for d in response.data]
    new_symbol = [d.new_symbol for d in response.data]
    d = {'Effective date': [date], 'Old symbol': old_symbol, 'New symbol': new_symbol}
    df = pd.DataFrame(data=d)
    return df

In [ ]:
# get and display symbol changes
symbol_changes = get_changes(response)
symbol_changes